In [ ]:
import os
import numpy as np
import pandas as pd
import pickle

from skimage.measure import compare_psnr
from sklearn.preprocessing import StandardScaler

import utils
from modules import RotationLSH, IdentityLSH, PcaLSH, RandomForestClustRegressor, PictureResolver

In [ ]:
model_names = ['./trained_models/example_model.pkl']

data_dir_test = '../data/Set14'
test_paths = os.listdir(data_dir_test)

In [ ]:
results_df = pd.DataFrame(columns=test_paths)

for row_num, model_name in enumerate(model_names):
    with open(model_name, 'rb') as inp:
        model = pickle.load(inp)

    config = model['config']
    scaler = model['scaler']
    lsh = model['lsh']
    rf = model['rf']
    
    s = config['s']
    augmented = config['augmented']
    
    pr = PictureResolver(rf, lsh, scaler)
    
    for param in ['s', 'n_samples', 'augmented', 'reg_type', 'min_samples_leaf', 'alpha']:
        results_df.loc[row_num, param] = config[param]
    
    lsh_type = config['lsh_type']
    if lsh_type == 'identity':
        lsh_type_full = 'identity'
    elif lsh_type == 'rotation':
        lsh_type_full = 'rotation_binary={}'.format(config['binary'])
    elif lsh_type == 'pca':
        lsh_type_full = 'pca_{}'.format(config['n_components'])
        
    results_df.loc[row_num, 'lsh_type'] = lsh_type_full


    for test_path in test_paths:
        img_test = utils.load_image(os.path.join(data_dir_test, test_path))
        img_test = utils.crop_image(img_test, s)
        x = utils.resize(utils.resize(img_test, s), 1/s)

        y = pr.resolve(x, augmented=augmented)
        images, y = utils.crop_by_nan([img_test], y)
        img_test = images[0]

        results_df.loc[row_num, test_path] = compare_psnr(img_test, y, np.max(img_test) - np.min(img_test))
        
pic_columns = [col for col in results_df.columns if col.endswith('.png')]
results_df['psnr_avg'] = results_df[pic_columns].apply(np.mean, axis=1)

In [ ]:
results_df